# Analyze the TIME / ESS distribution 

Randomly choose 25 samples and evaluate the model's Time / ESS 

### Import the libraries 

In [1]:
# Standard library imports
import sys

import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['KERAS_BACKEND'] = 'tensorflow'

# Third-party library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import arviz as az
import timeit

import scipy.stats as stats
from keras.models import Model as Model_nn
from keras.models import Sequential, load_model
from keras.layers import Dense, Concatenate
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Input, Dense, Add

#Try with TinyDA
import tinyDA as tda
from scipy.stats import multivariate_normal
from scipy.stats import uniform
from itertools import product


# Local module imports
sys.path.append('../../')
sys.path.append('../../solver')
#sys.path.append('./src/InverseProblems')
#sys.path.append('./src/utils')
from utils import * 
from plotting import *
from random_process import *
from model import *

2024-09-28 18:00:58.737606: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Ray module not found. Multiprocessing features are not available


### Choose the 25 random samples

In [2]:
n = 25 
np.random.seed(2109)
random_samples = np.random.randint(0, 160, n)
random_samples

array([ 57,  32,  55,  69,   3,  86,  23,  91,  53,  66, 109, 144,  94,
        88, 127,  54,  89,   3,  94,   9, 118,  88,  20,  28,  54])

### Load the data and surrogate model 

In [4]:
# Extract test data for visualization or further processing
n_eig = 64
X_values = np.loadtxt('../../data/50-25-10/X_test_50resolution.csv', delimiter = ',')
y_values = np.loadtxt('../../data/50-25-10/y_test_50resolution.csv',delimiter = ',')

# Set the resolution of the model and the random field parameters.
resolution = (50, 50)
field_mean = 1
field_stdev = 1
lamb_cov = 0.1
mkl_fine = 64

model_fine = Model(resolution, field_mean, field_stdev, mkl_fine, lamb_cov)

# Define the sampling points.
x_data = y_data = np.array([0.1, 0.3, 0.5, 0.7, 0.9])
datapoints = np.array(list(product(x_data, y_data)))


def model_hf(x):
    model_fine.solve(x)
    return np.array(model_fine.get_data(datapoints)).reshape(25)

### Time / ESS noise 0.001 and multiplicative coefficient 

In [6]:
noise = 0.001
scaling = 0.015
n_iter =  55000
burnin = 5000
thin = 50

Times = []
Time_ESS = []
ESS = []
i = 1

# Define the prior distribution and the proposal (common to all samples)
x_distribution = stats.multivariate_normal(mean = np.zeros(64), cov = np.eye(64))
my_proposal = tda.CrankNicolson(scaling=scaling, adaptive=False, gamma = 1.01, period=100)

for sample in random_samples:
    print('Sample = ', sample)
    x_true = X_values[sample]
    y_true = y_values[sample]

    y_observed = y_true + np.random.normal(scale=noise,size=y_true.shape[0])

    # LIKELYHOOD
    cov_likelihood = noise**2 * np.eye(25)
    y_distribution = tda.GaussianLogLike(y_observed, cov_likelihood*10)  

    # initialise the Posterior
    my_posterior = tda.Posterior(x_distribution, y_distribution, model_hf)

    # RUN THE MCMC
    start = timeit.default_timer()
    samples = tda.sample(my_posterior, my_proposal, iterations=n_iter, n_chains=1, initial_parameters=np.zeros(64))
    end = timeit.default_timer()

    # Remove the burnin and sub-sample
    idata = tda.to_inference_data(samples, level='fine')
    idata = idata.sel(draw=slice(burnin, None, thin), groups="posterior")
    ess = az.ess(idata)

    #Compute the time
    t = end-start
    Times.append(t)

    # Compute the mean ESS on the 64 parameters
    e = np.mean([ess.data_vars['x'+str(i)].values for i in range(64)])
    ESS.append(e)

    #Compute Time / ESS
    Time_ESS.append(t/e)
    
    print('Time:', t, '   ESS: ', e, '   Time/ESS: ',t/e , '     ',i,'/', len(random_samples))
    i = i+1

# Save the results 
# Specify the folder path (assuming it already exists)
folder_path = './recorded_values'  

# Save the file in the specified folder
file_path = os.path.join(folder_path, 'FOM_time_ess_001.npy')
np.save(file_path, Time_ESS)
file_path = os.path.join(folder_path, 'FOM_Times_001.npy')
np.save(file_path, Times)
file_path = os.path.join(folder_path, 'FOM_ESS_001.npy')
np.save(file_path, ESS)

Sample =  57
Sampling chain 1/1


Running chain, α = 0.37: 100%|██████████| 1000/1000 [00:14<00:00, 71.39it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 14.040981708007166    ESS:  4.3523067271708324    Time/ESS:  3.226101143182605       1 / 25
Sample =  32
Sampling chain 1/1


Running chain, α = 0.40: 100%|██████████| 1000/1000 [00:17<00:00, 57.37it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 17.447242583002662    ESS:  3.512139116899356    Time/ESS:  4.9676968941952735       2 / 25
Sample =  55
Sampling chain 1/1


...

Time: 13.958871665992774    ESS:  4.694631210626059    Time/ESS:  2.973369161436489       23 / 25
Sample =  28
Sampling chain 1/1


Running chain, α = 0.41: 100%|██████████| 1000/1000 [00:14<00:00, 67.13it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 14.912174999975832    ESS:  4.5350750563854145    Time/ESS:  3.2881870342982293       24 / 25
Sample =  54
Sampling chain 1/1


Running chain, α = 0.37: 100%|██████████| 1000/1000 [00:14<00:00, 68.38it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 14.640204124996671    ESS:  4.181046747003364    Time/ESS:  3.501564323692287       25 / 25


### Time/ESS Higher Noise

In [7]:
noise = 0.01
scaling = 0.04
n_iter = 55000
burnin = 5000
thin = 50

Times = []
Time_ESS = []
ESS = []
i = 1

# Define the prior distribution and the proposal (common to all samples)
x_distribution = stats.multivariate_normal(mean = np.zeros(64), cov = np.eye(64))
my_proposal = tda.CrankNicolson(scaling=scaling, adaptive=False, gamma = 1.01, period=100)

for sample in random_samples:
    print('Sample = ', sample)
    x_true = X_values[sample]
    y_true = y_values[sample]

    y_observed = y_true + np.random.normal(scale=noise,size=y_true.shape[0])

    # LIKELYHOOD
    cov_likelihood = noise**2 * np.eye(25)
    y_distribution = tda.GaussianLogLike(y_observed, cov_likelihood)  

    # initialise the Posterior
    my_posterior = tda.Posterior(x_distribution, y_distribution, model_hf)

    # RUN THE MCMC
    start = timeit.default_timer()
    samples = tda.sample(my_posterior, my_proposal, iterations=n_iter, n_chains=1, initial_parameters=np.zeros(64))
    end = timeit.default_timer()

    # Remove the burnin and sub-sample
    idata = tda.to_inference_data(samples, level='fine')
    idata = idata.sel(draw=slice(burnin, None, thin), groups="posterior")
    ess = az.ess(idata)

    #Compute the time
    t = end-start
    Times.append(t)

    # Compute the mean ESS on the 64 parameters
    e = np.mean([ess.data_vars['x'+str(i)].values for i in range(64)])
    ESS.append(e)

    #Compute Time / ESS
    Time_ESS.append(t/e)
    
    print('Time:', t, '   ESS: ', e, '   Time/ESS: ',t/e )

# Save the results 
# Specify the folder path (assuming it already exists)
folder_path = './recorded_values'  # Replace with your actual path

# Save the file in the specified folder
file_path = os.path.join(folder_path, 'FOM_time_ess_01.npy')
np.save(file_path, Time_ESS)
file_path = os.path.join(folder_path, 'FOM_Times_01.npy')
np.save(file_path, Times)
file_path = os.path.join(folder_path, 'FOM_ESS_01.npy')
np.save(file_path, ESS)

Sample =  57
Sampling chain 1/1


Running chain, α = 0.37: 100%|██████████| 1000/1000 [00:21<00:00, 47.58it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 21.24005325001781    ESS:  5.134508327768005    Time/ESS:  4.136725835100741
Sample =  32
Sampling chain 1/1


Running chain, α = 0.36: 100%|██████████| 1000/1000 [00:19<00:00, 51.41it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 19.472266500000842    ESS:  5.603682219292248    Time/ESS:  3.4749055599480823
Sample =  55
Sampling chain 1/1


...

Time: 18.478969958989182    ESS:  4.829065942951579    Time/ESS:  3.826613713146901
Sample =  69
Sampling chain 1/1


Running chain, α = 0.32: 100%|██████████| 1000/1000 [00:17<00:00, 56.75it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 17.639678750012536    ESS:  4.594724224147746    Time/ESS:  3.839115883670785
Sample =  3
Sampling chain 1/1


Running chain, α = 0.41: 100%|██████████| 1000/1000 [00:18<00:00, 53.77it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 18.61447958298959    ESS:  5.217153446045959    Time/ESS:  3.5679379139399017
